# Classifiers
This notebook evaluates the accuracy of several classifiers in distinguishing among treatment-induced phenotypes.

### imports

In [ ]:
%run notebook_imports
%run notebooks/matplotlib_nb

In [ ]:
# 3rd party libraries
import pandas as pd
import numpy as np
import scipy.stats
from collections import defaultdict 
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import sklearn
from tqdm import tqdm
import pickle
from os import listdir, mkdir

In [ ]:
from utilities import PartialPca
from utilities.paths import path
from utilities.constants import column_order, index_order, CONC, TREAT
from utilities.data_cleaning import label_data, format_column_name, format_dataframe_columns, normalise_data, clean_data

### parameters

In [ ]:
balance_classes = True # whether to balance class numbers
savefig = True # whether to save figures
showfig = True # whether to show figures in the notebook
apply_log_transform = False # apply log transformation to the data
# how many PCA components to keep. if None, data is not transformed.
n_kept_components = None

### data loading

In [ ]:
data_path = path.data_classifiers
files_list = listdir(data_path)
dataset_list=[filename for filename in files_list if filename.endswith('.csv')]

In [ ]:
dataframes = [pd.read_csv(data_path+csv,encoding='utf-8')
              for csv in dataset_list]

In [ ]:
# removing Weighted_Relative_Moment_Inertia
# high frequency of nan
dataframes = [d.drop(columns=['Weighted_Relative_Moment_Inertia']) for d in dataframes]

### other

In [ ]:
# for filename
PCA_string = str(n_kept_components)+'components'
param_string = (''
               + PCA_string + '-'
               + ('balanced' if balance_classes else 'unbalanced')
               + '_')
plots_dir = path.plots + 'classifiers/'
try: mkdir(plots_dir)
except FileExistsError: pass

# data cleaning and proper labeling

In [ ]:
def rename_numbers_column(df):
    """Column 'col' should be called 'number'"""
    cols = df.columns
    cols = [colname if colname != 'col' else 'number' for colname in cols]
    df.columns = cols
    return df
dataframes = [rename_numbers_column(d) for d in dataframes]

In [ ]:
dataframes = [d.dropna() for d in dataframes]
dataframes = [format_dataframe_columns(d) for d in dataframes]
dataframes = [label_data(d) for d in dataframes]

## Data analysis

### subsetting
keep only some treatments, keep only nanoparticles with 0 concentration (no treatment) and maximum concentration (most nanoparticle effect)

In [ ]:
kept_treatments = [#'FCCP Control', 
                   'Si-CNPPV','Si-P3', 
                   'Si-P4',  'PP-CNPPV',    
                   'PP-P3',
                   'PP-P4', ]
kept_concentrations = ['300ug/mL','0 ug/mL', 'control']
def subset_data(dataframe):
    keep = dataframe[TREAT].isin(kept_treatments) & dataframe[CONC].isin(kept_concentrations)
    return keep

In [ ]:
clean_dataframes = [d.loc[subset_data(d),:] for d in dataframes]

## division in classes
one class for each nanoparticle type (PP and Si), and a class for no treatment

In [ ]:
def relabel_classes(cl):
    if cl.startswith('PP'):
        return 'PP CPNs'
    elif cl.startswith('Si'):
        return 'Si CPNs'
    else:
        return cl

In [ ]:
def class_creation(dataframe):
    classes = pd.Series(['' for _ in range(dataframe.shape[0])])

    for t in kept_treatments:
        classes[(dataframe[TREAT] == t).values] = relabel_classes(t)

    treated = (dataframe[CONC] == '300ug/mL').values
    classes[(dataframe[CONC] == '0 ug/mL').values] = 'Untreated'

    dataframe['CLASS'] = classes.tolist()
    return dataframe

In [ ]:
clean_dataframes = [class_creation(d) for d in clean_dataframes]

drop columns that are not measurements, only keep numerical measurements and class column

In [ ]:
def drop_columns(dataframe):
    for f in ['Row', CONC, TREAT, 'Target']:
        try:
            dataframe = dataframe.drop(columns=f)
        except ValueError:
            pass
    return dataframe

In [ ]:
clean_dataframes = [drop_columns(d) for d in clean_dataframes]

# data normalization
z-scoring, optionally logarithm

In [ ]:
def normalise_df(dataframe):
    classes = dataframe['CLASS']
    #select only numeric data
    numeric = dataframe._get_numeric_data()

    #apply transformation
    numeric = numeric - numeric.mean()
    numeric = numeric / numeric.std()

    dataframe = numeric
    # class information back in
    dataframe['CLASS'] = classes.tolist()
    return dataframe


In [ ]:
clean_dataframes = [normalise_df(d) for d in clean_dataframes]

## aggregate individual datasets

In [ ]:
cleandata = pd.concat(clean_dataframes)
del clean_dataframes

otpional logarithm

In [ ]:
if apply_log_transform:
    cleandata = cleandata.apply(np.log)
    # if you apply log these features contain numbers <=0, so have to be removed
    cleandata = cleandata.drop(['WMOI Cell', 'Moment Cell'])

## balanced class counts

In [ ]:
# class counts still unbalanced!
class_counts = cleandata['CLASS'].value_counts()
class_counts

In [ ]:
if balance_classes:
    # balance the classes.
    max_per_class = min(class_counts)
    selection = [] # here we'll put the rows to keep
    for klass in class_counts.index: #iterate on all classes
        # bool selection of all elements in a class
        this_class = cleandata['CLASS'] == klass
        # turn into an int selection
        this_class_indices = np.where(this_class.values)[0]
        # if  you have more elements than `max_per_class` in this class
        if this_class.sum() > max_per_class:
            # choose randomly `max_per_class` elements
            choice = np.random.choice(this_class_indices,size=max_per_class,replace=False)
            # add them to the selection to keep
            selection += choice.tolist()
        else: # otherwise
            # keep all elements
            selection += this_class_indices.tolist()
    # subselect
    cleandata = cleandata.iloc[selection,:]

In [ ]:
# check that it worked
class_counts = cleandata['CLASS'].value_counts()
class_counts

In [ ]:
# update classes vector, we'll need it later
classes = cleandata['CLASS']
#remove classes from dataframe to do dimensionality reduction
cleandata = cleandata.drop(columns='CLASS')

## dimensionality reduction
apply a PCA to the data, keep first N components

In [ ]:
PCA = PartialPca.PartialPCA(n_components = cleandata.shape[1])

In [ ]:
PCA.fit(cleandata.values)
results = PCA.transform(cleandata.values).astype(np.dtype('float64'))

look at the loadings of the PCA

In [ ]:
# extract eigenvectors
f = pd.DataFrame(np.vstack([x.astype(float) for x in PCA.eig_vec]).T)
f.index = cleandata.columns
f_abs=f.abs()
# format feature names
f_abs.index = [format_column_name(x) for x in f_abs.index]

In [ ]:
plt.close('all')
fig, ax = plt.subplots(figsize=(12,12))
sns.heatmap(f_abs.round(2),  cmap='OrRd', linewidths=2, square=True, annot=True, fmt='0.01f', annot_kws={"size": 7},  cbar = False, 
            cbar_kws={"shrink": 0.1}, robust=True)
plt.subplots_adjust(
 left  = 0.25 , right = 0.95
)
ax.set_xlabel('Principal Components (PC)')
ax.set_ylabel('Features')
if savefig: fig.savefig(plots_dir+param_string+'PCA_weights.png', dpi=600)

explained variance plot

In [ ]:
fig,ax = plt.subplots(figsize =(4, 3))
var = PCA.explained_variance()
ax.scatter(list(range(len(var))), np.cumsum(var), c=var,marker="8",cmap="OrRd",s= 10,   linewidths=0.4, edgecolors="Red")
ax.bar(list(range(len(var))), var, color='firebrick', edgecolor='Red')
ax.set_title('PCA Explained Variance')
ax.set_xlabel('Component #')
ax.set_ylabel('Explained variance (%)')
fig.tight_layout()
if savefig: fig.savefig(plots_dir+param_string+'PCA_explained_variance.png', dpi=600)
if showfig: fig.show()

In [ ]:
if n_kept_components is not None: # transform data
    # take only first `n_kept_components` components
    transformed_data = pd.DataFrame(results[:,:n_kept_components])
    transformed_data.columns = ['PCA_comp_#'+str(x) for x in transformed_data.columns]
else: # don't do PCA at all
    transformed_data = cleandata._get_numeric_data()
# reinsert classes column
transformed_data['CLASS'] = classes.tolist()

first 4 components pairplots

In [ ]:
if n_kept_components is not None:
    plt.close('all')
    sns.set_style("ticks", {"xtick.major.size": 3, "ytick.major.size": 3})
    g = sns.pairplot(transformed_data[['PCA_comp_#0', 'PCA_comp_#1', 'PCA_comp_#2', 'PCA_comp_#3','CLASS']],
                     hue='CLASS',kind= 'scatter', 
                     diag_kind='kde', palette="inferno", size=1.4, aspect=1.2, markers = "o",  
                     plot_kws=dict(s=1, linewidth=0, alpha=0.4, #MarkerEdgeAlpha= 1, edgecolor = 'red'
                                  ) ,  
                     diag_kws=dict(linewidth=1, shade=True, alpha=0.2, vertical=False, shade_lowest = True))
    for ax in g.axes[0,:]:
        ax.get_yaxis().set_label_coords(-0.3,0.5)

    g.axes[0,0].set_ylabel('PC1')
    g.axes[1,0].set_ylabel('PC2')
    g.axes[2,0].set_ylabel('PC3')
    g.axes[3,0].set_ylabel('PC4')
    g.axes[3,0].set_xlabel('PC1')
    g.axes[3,1].set_xlabel('PC2')
    g.axes[3,2].set_xlabel('PC3')
    g.axes[3,3].set_xlabel('PC4')

    for ax in g.axes.flatten():
        ax.set_xscale("symlog", nonposy='clip')
        ax.set_yscale("symlog", nonposy='clip')
        ax.tick_params(labelsize=5.5, pad=4,
                       zorder =10)

    g._legend.set_title("Treatments:")
    for lh in g._legend.legendHandles: 
        lh.set_alpha(1)
        lh._sizes = [30] 

    if savefig: plt.savefig(plots_dir+param_string+'pairplot.png',dpi=600)

    if showfig: plt.show()

## classification
let's try a set of simple go-to algorithms: random forest, KNN and MLP

In [ ]:
available_classes = set(classes)

In [ ]:
#shuffle data
# dataset = transformed_data.sample(frac=1)
transformed_data = transformed_data.dropna()

# split into features and target (classes)
X = transformed_data._get_numeric_data().values
y = transformed_data['CLASS'].values

In [ ]:
#imports
from sklearn.model_selection import KFold
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from collections import defaultdict
from types import SimpleNamespace
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

def single_class_accuracy(true_val,pred_val,klass):
    """tests accuracy for a single class as opposed to overall accuracy"""
    in_class = true_val==klass
    correct = true_val[in_class]==pred_val[in_class]
    return correct.sum()/correct.size
def remove_nans(array):
    array = np.array(array)
    return array[np.isfinite(array)]

list algorithms to use together with parameters

In [ ]:
randomforest = SimpleNamespace()
randomforest.model = RandomForestClassifier
randomforest.name = 'RandomForest'
randomforest.params = {'n_jobs':4,'n_estimators':40}

knn = SimpleNamespace()
knn.model = KNeighborsClassifier
knn.name = 'KNN'
knn.params = {'n_jobs':4}

mlp = SimpleNamespace()
mlp.model = MLPClassifier
mlp.name = 'ANN'
mlp.params = {'alpha':1}

all_models = [randomforest,knn,mlp]


establish baseline

In [ ]:
strategies = ['most_frequent', 'stratified', 'prior', 'uniform']
baseline_results = dict()
for strategy in strategies:
    baseline = DummyClassifier(strategy=strategy)
    baseline.fit(X,y)
    baseline_prediction = baseline.predict(X)
    baseline_score = accuracy_score(y, baseline_prediction)
    baseline_results[strategy] = baseline_score
baseline_max = max(baseline_results.values())
baseline_results

In [ ]:
class_counts_dict = {idx:va for idx,va in zip(class_counts.index,class_counts.values)}

In [ ]:
def test_kfold(algorithm):
    kf = KFold(n_splits=20, shuffle=True)
    kf.get_n_splits(X)

    scores = defaultdict(list)
    for train_index, test_index in kf.split(X):
        classifier =  algorithm.model(**algorithm.params)
        classifier.fit(X[train_index], y[train_index])
        model_prediction=classifier.predict(X[test_index])
        scores['all'].append(accuracy_score(y[test_index], model_prediction))
        for klass in available_classes:
            scores[klass].append(single_class_accuracy(y[test_index], model_prediction,klass)) 
    return scores

In [ ]:
def plot_kfold_results(algorithm,scores):
    plt.close('all')
    fig,ax = plt.subplots(figsize = (6, 4))
    bins = np.linspace(0,1,num=50,endpoint=True)
    label = 'All cells'
    ax.hist(scores['all'], label=label, alpha=.7, bins=bins)
    score_mean = np.mean(scores['all'])
    ax.axvline(score_mean, label = 'Global accuracy',
               color = 'green', alpha=.4)
    for klass in available_classes:
        label = klass
        plt.hist(remove_nans(scores[klass]), label=label, alpha=.2, bins=bins)
    ax.axvline(baseline_max, label='Baseline',
               color = 'red', alpha=.4)
    ax.set_xlim(0,1)
    ax.set_ylim(bottom=0)
    bbox_inches="tight"
    ax.legend(loc=2, frameon =1, fancybox =1, labelspacing = 0.5)
    ax.set_xlabel('Accuracy (a.u.)')
    ax.set_ylabel('KFolds Count')
    ax.set_title('Accuracy single cells classification by treatment on all folds')
    fig.tight_layout()
    if savefig: fig.savefig(plots_dir+param_string+algorithm.name+'_accuracy.png')
    if showfig: fig.show()

In [ ]:
# compute test results:
for M in all_models:
    scores = test_kfold(M)
    plot_kfold_results(M,scores)
    print(M.name,'accuracy:', np.round(np.mean(scores['all']),3))